<a href="https://www.kaggle.com/code/brenodahora/treinamento-yolo-para-placas-de-licenciamento?scriptVersionId=209474688" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Código de treinamento do YOLO para detecção de placas de licenciamento

## Instalação de dependências necessárias

Este comando instala as bibliotecas necessárias para o funcionamento do código em suas versões adequadas:

- `ultralytics==8.2.66`: Biblioteca que contém o framework para utilizar os modelos YOLO.
- `ipywidgets==8.1.3`: Ferramenta que permite a interação visual em Jupyter Notebooks, compatível com o **wandb=0.17.5**.
- `wandb==0.17.5`: Biblioteca para o rastreamento e visualização de experimentos de machine learning.

A opção `-q` reduz a verbosidade do processo de instalação para evitar muitas mensagens no notebook.

In [ ]:
!pip install ultralytics==8.2.66 ipywidgets==8.1.3 wandb==0.17.5 -q

## Importação de bibliotecas essenciais

As bibliotecas a seguir são importadas para executar diferentes etapas do projeto:

- `ultralytics.YOLO`: Importa a classe principal do YOLO para carregar, treinar e avaliar modelos de detecção de objetos.
- `os`: Fornece funções para interagir com o sistema operacional, como manipulação de arquivos e diretórios.
- `json`: Permite manipular dados no formato JSON, comum para armazenar configurações ou resultados.
- `wandb`: Integração com a plataforma Weights & Biases para registrar e visualizar experimentos de aprendizado de máquina.
- `shutil`: Oferece operações de alto nível em arquivos e coleções de arquivos, como cópias e movimentações de diretórios.

Essas bibliotecas são fundamentais para lidar com modelos, arquivos de configuração e o rastreamento dos experimentos.


In [ ]:
from ultralytics import YOLO
import os
import json
import wandb
import shutil

## Cópia de arquivos e pastas

Necessário quando o treinamento atinge o tempo limite de sessão (12 horas) e é necessário retomar o treinamento em uma nova sessão.

- Os arquivos do treinamento anterior devem ser carregados no diretório de entrada (*input*) do kaggle e então eles serão copiados para o diretório de trabalho (*working*).

In [ ]:
def copy_files_and_folders(src, dst):
    try:
        for item in os.listdir(src):
            src_item = os.path.join(src, item)
            dst_item = os.path.join(dst, item)
            
            if os.path.isdir(src_item):
                if not os.path.exists(dst_item):
                    os.makedirs(dst_item)

                copy_files_and_folders(src_item, dst_item)
            else:
                shutil.copy2(src_item, dst_item)
        
        print(f"Contents of '{src}' copied to '{dst}' successfully.")
    except Exception as e:
        print(f"Error: {e}")

# copy_files_and_folders('/kaggle/input/yolov5su-g', '/kaggle/working')

## Download do modelo YOLO a ser treinado

Nesta etapa é feito o download da versão do YOLO que servirá como base para o treinamento da nova classe.

In [ ]:
model = YOLO('yolov5su.pt')

## Configuração e inicialização do Weights & Biases (WandB)

Nesta célula, são realizados os passos para configurar e iniciar o rastreamento do treinamento no WandB:

1. **Carregamento de chave de API**:  
   - O arquivo `/kaggle/input/secrets/secrets.json` contém a chave de API do WandB.

2. **Autenticação no WandB**:  
   - A função `wandb.login` utiliza a chave de API para autenticar o usuário.

3. **Inicialização do projeto no WandB**:  
   - `wandb.init` configura um novo experimento ou continua um existente:
     - `resume="allow"`: Permite retomar o experimento caso ele já tenha sido iniciado anteriormente, caso seja necessário retomar o treinamento.

In [ ]:
with open('/kaggle/input/secrets/secrets.json') as f:
    secrets = json.load(f)
    
wandb.login(key=secrets['WB_API_KEY'])
wandb.init(
    project='License Plate Optimized Final',
    name='yolov5su-g',
    id="yolov5su-g",
    resume="allow"
)

## Treinamento do modelo YOLO

Nesta etapa, o modelo YOLO é treinado usando um conjunto de dados (*dataset*) personalizado:

- `model.train`: Método utilizado para iniciar o treinamento do modelo YOLO carregado na variável `model`.
  - `data='/kaggle/input/customlicenseplatedataset/customlicenseplatedataset/data.yaml'`:  
    Especifica o caminho para o arquivo `data.yaml`, que contém as configurações do dataset.  
    Este arquivo define:
    - O caminho para as imagens de treinamento e validação.
    - As classes de objetos a serem detectadas.
   
- O comando `resume=True` deve ser adicionado para retomar treinamentos interrompidos.

In [ ]:
results = model.train(
    data='/kaggle/input/customlicenseplatedataset/customlicenseplatedataset/data.yaml'
)

## Finalização da execução no WandB

- `wandb.finish()`: Finaliza a sessão atual do WandB.  
  - Essa função garante que todos os dados coletados, como métricas, gráficos e logs do treinamento, sejam salvos e enviados para o servidor do WandB.

In [ ]:
wandb.finish()

## Exemplos de arquivos de entrada

- `secrets.json`:

    ```
    {
        "WB_API_KEY": "xxxxxxxxxxxxxxx"
    }    ```


- `data.yaml`:

    ```
    path: /kaggle/input/customlicenseplatedataset/customlicenseplatedataset
    train: images/train
    val: images/val
    test: images/test

    names:
        0: License_Plate

    roboflow:
        workspace: roboflow-universe-projects
        project: license-plate-recognition-rxg4e
        version: 4
        license: CC BY 4.0
        url: https://universe.roboflow.com/roboflow-universe-projects/license-plate-recognition-rxg4e/dataset/4
    
    ```